In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 자기 google.exe 저장 주소 입력
# C:/Program Files (x86)/Google/Chrome/Application/chrome.exe 이 부분만 바꾸면 된다.
subprocess.Popen(r'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver = webdriver.Chrome('diver/chromedriver.exe', chrome_options=options)

driver.implicitly_wait(3)

# 링티 제로
url = 'https://www.coupang.com/vp/products/5688038203?itemId=9412000674&isAddedCart='
# 815
url = 'https://www.coupang.com/vp/products/5109971962?itemId=6971621377&isAddedCart='
# 부르르 제로 콜라
url = 'https://www.coupang.com/vp/products/2358334857?itemId=4093099810&vendorItemId=72077082112&isAddedCart='
# test data
url = 'https://www.coupang.com/vp/products/6778883011?vendorItemId=83142028803&sourceType=HOME_GW_PROMOTION&searchId=feed-9d07ae1c003e479c84b9382e8fec5ae7-gw_promotion&isAddedCart='
driver.get(url)

C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_468\2201847069.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('diver/chromedriver.exe', chrome_options=options)
C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_468\2201847069.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('diver/chromedriver.exe', chrome_options=options)


- By.ID	태그의 id값으로 추출
- By.NAME	태그의 name값으로 추출
- By.XPATH	태그의 경로로 추출
- By.LINK_TEXT	링크 텍스트값으로 추출
- By.PARTIAL_LINK_TEXT	링크 텍스트의 자식 텍스트 값을 추출
- By.TAG_NAME	태그 이름으로 추출
- By.CLASS_NAME	태그의 클래스명으로 추출
- By.CSS_SELECTOR	css선택자로 추출

## 쿠팡 크롤링 주의점
- 가끔 주소에 js라는 값이 들어있는데 COPY한 주소의 js 앞 부분이 ' '(띄어쓰기)가 되어있다.
- 이 때 ' ' 대신 '.'을 넣어주자.
    - ex) 'sdp-review__article__list__review__content js_reviewArticleContent'
        - 'sdp-review__article__list__review__content.js_reviewArticleContent'

In [18]:
# 코딩 시작

In [57]:
# 리뷰 크롤링 함수 생성


def review_crawling_data(lst : list) :
    # 한 페이지에 나타나는 리뷰 개수 만큼 반복
    review_length = len(driver.find_elements(By.CLASS_NAME , "sdp-review__article__list.js_reviewArticleReviewList"))+1
    for rev in range(1, review_length) :
        try :
            # 리뷰는 본문을 적는 메인 리뷰와 짧은 내용만 적는 서브 리뷰로 나뉜다.
            # 리뷰는 두 리뷰 모두 존재하는 경우 , 메인 리뷰만 존재하는 경우, 서브 리뷰만 존재하는 경우로 나뉜다.
            # 이 때, 둘 다 존재할 경우 예외처리는 통과된다. 그러나 둘 중 하나만 존재할 경우 예외를 발생시킨다.
            data_main = driver.find_element(By.XPATH 
                                      , f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/article[{rev}]/div[3]').text
            data_sub = driver.find_element(By.XPATH 
                                      , f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/article[{rev}]/div[4]/div').text
            # 예외 발생
            # 메인/서브 리뷰가 하나만 존재할 경우 data_sub에 맛 만족도 평가가 들어가진다.
            # 이 때 data_sub를 검사하여 예외를 발생시킨다.
            if '맛 만족도' in data_sub :
                raise Exception
            
        except Exception as e:
            # 리뷰 내용이 존재하지 않을 경우 data_main값에 신고하기 가 들어가진다.
            # 이 때 신고하기가 아닌 경우만 데이터를 가져온다.
            if data_main != '신고하기' :
                lst.append(data_main)
                # 리뷰 크롤링 완료 확인
                print(f'review_0{rev}\t' , end = '')
            else :
                print('None review data\t' , end = '')
                pass
            # print('예외발생')
        else :
            # try 통과시 메인/서브 리뷰 모두 저장
            lst.append(data_main + data_sub)
            # 리뷰 크롤링 완료 확인
            print(f'review_0{rev}\t' , end = '')


In [35]:
# 리뷰 개수 확인 함수

import re
def review_total() :
    text = driver.find_element(By.XPATH , '//*[@id="btfTab"]/ul[1]/li[2]/span').text
    # [\W] = [^0-9a-zA-Z] : 0~9 a~z A~Z 문자를 가져옵니다.
    return int(re.sub('[\W]' , '' , text))

In [68]:
# 쿠팡 리뷰 크롤링 함수 생성
# wait_time : 대기 시간

def crawling_page_load_and_input_review_data(object_name : str) :
    
    driver.find_element(By.XPATH , '//*[@id="btfTab"]/ul[1]/li[2]').click()
    
    globals()['result'] = []
    rev_tl = review_total()
    if rev_tl >= 1500 :
        count = 30
    elif rev_tl <= 50 :
        count = 1
    else :
        count = int(rev_tl / 5 / 10)
    print(count)
        
    wait_time = 3
    for lop in range(count) :
        # 로딩 완료 확인
        WebDriverWait(driver, wait_time).until(
                        EC.presence_of_element_located(
                            (By.XPATH,'//*[@id="container"]')
                )
            )
        # 리뷰 페이지 수 만큼 루프를 돌리기 위해 리뷰 페이지 수의 길이를 잰다.
        # 리뷰 페이지 수는 '오른쪽 화살표' 버튼 클릭시 바뀌게된다.
        page_btn_length = len(driver.find_elements(By.XPATH , '//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button'))
        for page in range(2, page_btn_length) :
            WebDriverWait(driver, wait_time).until(
                        EC.visibility_of_all_elements_located(
                            (By.CLASS_NAME,'sdp-review__article__list.js_reviewArticleReviewList')
                )
            )
            # 페이지를 출력한다.
            # 마지막 페이지에는 리뷰 페이지 번호란이 존재하지 않는다.
            # 위 사실을 이용해서 마지막 페이지를 확인한다.
            # 해당 주소에 리스트 값이 존재할 경우 마지막 페이지가 아니다.
            # 해당 주소에 리스트 값이 존재하지 않을 경우 마지막 페이지가 된다.
            if driver.find_elements(By.CLASS_NAME , "sdp-review__article__page.js_reviewArticlePagingContainer") != [] :
                page_count = driver.find_elements(By.CLASS_NAME , "sdp-review__article__page.js_reviewArticlePagingContainer > button")[page-1].text
                print(f'page - {page_count}')
                # 리뷰 크롤링
                review_crawling_data(result)
                print()
                time.sleep(1)
                # 페이지 이동 버튼
                driver.find_elements(By.CLASS_NAME , "sdp-review__article__page.js_reviewArticlePagingContainer > button")[page].click()
                time.sleep(1)

        print('last page')
        print('크롤링 종료')
        sample_ringteazero_review = pd.DataFrame({'review' : result})
        sample_ringteazero_review.to_excel(f'crawling_test/{object_name}_coupang_crawling.xlsx' , index = False)



In [64]:
len(driver.find_elements(By.XPATH , '//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button'))

5

In [66]:
driver.find_elements(By.CLASS_NAME , "sdp-review__article__page.js_reviewArticlePagingContainer")

[<selenium.webdriver.remote.webelement.WebElement (session="f3c8b6a3a598c941da84db42455ea61e", element="6287825c-2c02-4a36-bf2f-96dfd844e9bc")>]

In [44]:
review_total()

22

In [52]:
driver.find_elements(By.CLASS_NAME , "sdp-review__article__page.js_reviewArticlePagingContainer")

[<selenium.webdriver.remote.webelement.WebElement (session="238f583c7602db758a842e355c3c9ee5", element="6b88e8eb-3cb0-4e54-9bb4-b5c285e733a4")>]

In [70]:
# 함수 실행
# 함수에는 상품 영문 이름 입력

crawling_page_load_and_input_review_data('test')

# crawling_page_load_and_input_review_data('ringteazero')
# crawling_page_load_and_input_review_data('815_cola')
# crawling_page_load_and_input_review_data('bururu_zero_cola')


1
page - 1
review_01	review_02	review_03	review_04	review_05	
page - 2
review_01	review_02	None review data	review_04	None review data	
page - 3
None review data	None review data	None review data	None review data	None review data	
last page
크롤링 종료


In [31]:
# 리뷰 개수 확인
len(result)

1500